### Merge values in one column based of the value in other column

We have some deals with multiple products. Create another dataset with Deal Id, all Products as csv and their total Amount


    Deal Id                 Products    Amount                             Deal Id                Products        Amount
    0063a00000h6PtsAAE      RHEL        50              ----------->       0063a00000h6PtsAAE     RHEL,ANSIBLE    100
    0063a00000h6PtsAAE      ANSIBLE     50

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder\
    .appName('Day3Spark')\
    .master('local')\
    .getOrCreate()

In [3]:
df = spark.read.format('csv')\
    .option('header','true')\
.load("interview_data.csv")

In [4]:
#df.printSchema()

In [5]:
import re
from pyspark.sql import functions as F
df = df.select([F.col(col).alias(col.replace('(', '')) for col in df.columns])
df = df.select([F.col(col).alias(col.replace(')', '')) for col in df.columns])

In [6]:
df.printSchema()

root
 |-- Deal ID: string (nullable = true)
 |-- Products: string (nullable = true)
 |-- Industry: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Super Region: string (nullable = true)
 |-- Start Date: string (nullable = true)
 |-- Term in Days: string (nullable = true)
 |-- Amount $: string (nullable = true)



In [7]:
df2 = df.select("Deal ID","Products","Amount $")

In [8]:
df2.show()

+------------------+--------+------------------+
|           Deal ID|Products|          Amount $|
+------------------+--------+------------------+
|0063a00000h6PtsAAE|    RHEL|       1327.912875|
|0063a00000h6PtsAAE|    RHEL| 523.7066249999998|
|0063a00000hDX0rAAG| ANSIBLE|         88730.625|
|0063a00000hDkUDAA0| ANSIBLE|         88730.625|
|0063a00000iER6TAAW|    RHEL|           31437.5|
|0063a00000iEplCAAS| ANSIBLE|           27232.5|
|0063a00000iEq3iAAC|    RHEL|       5148.323575|
|0063a00000iEq3jAAC|    RHEL|         2782.1911|
|0063a00000iEq3jAAC|    RHEL| 3166.822450000001|
|0063a00000iEq3jAAC|    RHEL|        425.814225|
|0063a00000iEq3jAAC|    RHEL|      19223.383675|
|0063a00000iEq3jAAC|    RHEL|        3648.79955|
|0063a00000iEq3jAAC|    RHEL| 9.540249999999986|
|0063a00000gP8HzAAK|    RHEL|          633.7725|
|0063a00000h6uWwAAI|    RHEL|        2102.28465|
|0063a00000h6uWwAAI|    RHEL|       1159.923125|
|0063a00000h9ALpAAM|TRAINING|    2492553.587275|
|0063a00000h9ALpAAM|

In [9]:
from pyspark.sql import functions as func

In [10]:
(df2
    .groupBy("Deal ID")
    .agg(func.sum("Amount $"),func.collect_list("Products"))
    .show())

+------------------+------------------+----------------------+
|           Deal ID|     sum(Amount $)|collect_list(Products)|
+------------------+------------------+----------------------+
|              null|              null|                    []|
|0063a00000gN5cQAAS|     2184914.37475|  [CLOUD INFRASTRUC...|
|0063a00000gN6UVAA0|         4204800.0|  [CONSULTING, CONS...|
|0063a00000gN98hAAC|       5584.240625|  [RHEL, RHEL, RHEL...|
|0063a00000gNA4bAAG|        14602.7825|                [RHEL]|
|0063a00000gNAmuAAG| 36872.12100000001|          [RHEL, RHEL]|
|0063a00000gNCFuAAO| 42275.73585000001|  [RHEL, RHEL, RHEL...|
|0063a00000gNDcwAAG|40421.072824999996|    [RHEL, RHEL, RHEL]|
|0063a00000gNEHRAA4|          17538.53|  [RHEL, RHEL, RHEL...|
|0063a00000gNEKlAAO|      474563.54865|    [RHEL, MIDDLEWARE]|
|0063a00000gNGNLAA4|32442.637075000006|          [RHEL, RHEL]|
|0063a00000gNHHwAAO| 9497.140300000001|  [RHEL, RHEL, RHEL...|
|0063a00000gNJTqAAO|       9012.035575|    [RHEL, RHEL,